In [1]:
import numpy as np
import random
from sympy import isprime
from fractions import Fraction

import sys
from tqdm import tqdm

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator

from utils import bin_to_int

service = QiskitRuntimeService()

In [8]:
class QDSimulator:
    def __init__(self, 
                 lower_bound=int(1e1), 
                 upper_bound=int(1e3), 
                 message="hello world",
                 num_Qbits=5,
                 verbose=False,
                 ):
        # initialize lower and upper bound for prime numbers
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        # string message
        self.message = message
        # num Qbits
        self.n = num_Qbits
        # print stuff for debugging purposes
        self.verbose = verbose

        print("QDSimulator intitialized!")

    def InitializeEncryptionKeys(self):
        """
            Generates encoding (encoding_key) and decoding (decoding_key) keys given two distinct 
            prime numbers (p and q)
        """
        self._generate_primes()
        self._generate_encoding_key()
        self._generate_decoding_key()

    def _generate_primes(self):
        """
            Chooces two distinct random prime numbers (p and q) between the given lower (lower_bound)
            and upper (upper_bound) bounds and computes their product (N=p*q) and the group order
            (group_order = (p-1)*(q-1))
        """

        # construct a list of primes between lower_bound and upper_bound
        primes = [i for i in range(self.lower_bound,self.upper_bound) if isprime(i)]
        # pick two distinct random primes from list
        p, q = random.sample(primes, 2)
        self.p, self.q = p, q
        self.N, self.group_order = p*q, (p-1)*(q-1)

        print(f"\nGenerated random prime numbers p={p} and q={q}, and their product N={self.N}")
    
    def _generate_encoding_key(self):
        """
            Generates the encoding key by finding a number that has no factor in common with
            group_order = (p-1)*(q-1). Picks random candidates that are smaller than group_order
            and checks their GCD via the Euclidean algorithm. If GCD=1, candidate is chosen. 
            
            The encoding key should be shared with others.
        """

        found_encoding_key = False
        
        # loop over encoding_key candidates. Add loop counter to avoid infinite while loop
        loop_counter1 = 0
        while (not found_encoding_key and loop_counter1 < 100):
            # candidate for encoding key c
            encoding_key_candidate = random.randint(self.lower_bound, self.group_order-1) # lower bound needed?
            # re-set f and c for next candidate
            c = encoding_key_candidate
            f = self.group_order

            # apply the Euclidean algorithm using loop. Add loop counter to avoid infinite while loop
            loop_counter2 = 0
            while (c > 0 and loop_counter2 < 100):
                # c from last iteration
                c_0 = c
                # Euclidean algorithm
                f, c = c, f - int(f/c)*c
                loop_counter2+=1
            
            # if GCD = 1, encoding key is c from last iteration
            if c_0 == 1:
                found_encoding_key = True
                self.encoding_key = encoding_key_candidate

            loop_counter1+=1

        if found_encoding_key:
            print(f"\nEncoding key generated c={self.encoding_key} (with (p-1)*(q-1)={self.group_order})")
        else:
            sys.exit(f"Could not generate an encoding key after {loop_counter1} attempts. Try increasing maximum allowed attempts")

    def _generate_decoding_key(self):
        """ 
            Generates the decoding key by finding the inverse modulo of the encoding key
                decoding_key * encoding_key = 1 (mod (p-1)(q-1))
            Do this by setting GCD=1 and applying the Eculidean algorithm backwards ... 

            The decoding key should not be shared with others.
        """

        f = self.group_order
        c = self.encoding_key

        found_decoding_key = False
        it = self.N 
        print("\nGenerating decoding key ...")
        for j in tqdm(range(it)):
            for k in range(-it,it):
                if (j*f+k*c == 1):
                    found_decoding_key = True
                    break
            if found_decoding_key:
                break
        if not found_decoding_key:
            sys.exit("Could not generate a decoding key. Try increasing searching parameter")

        d = k % f
        if (remainder := (c*d) % self.group_order) != 1:
            sys.exit(f"Decoding key check failed, incorrect key generated ...")

        self.decoding_key = d
        print(f"..Done. Check pased. Decoding key d={d}")
    

    def EncryptMessage(self):
        """ 
            Add description
        """

        ascii_message = np.array([ord(char) for char in self.message])
        if ascii_message.any() > self.N:
            sys.exit(f"N to small. N is {self.N} and the largest ascii code is {max(ascii_message)}")

        encoding_key = self.encoding_key
        N = self.N
        encrypted_message = []
        for ascii_code in ascii_message:
            encrypted_ascii_code = pow(int(ascii_code), encoding_key, N)
            encrypted_message.append(encrypted_ascii_code)
        self.encrypted_message = encrypted_message

        print(f"\nMessage has been encrypted to {encrypted_message}")
    
    def DecryptMessage(self):
        """ 
            Add description
        """

        decoding_key = self.decoding_key
        N = self.N
        decrypted_ascii_message = []
        for encrypted_ascii_code in self.encrypted_message:
            decrypted_ascii_code = pow(encrypted_ascii_code, decoding_key, N)
            decrypted_ascii_message.append(decrypted_ascii_code)
        
        #decrypted_message = [chr(ascii_code) for ascii_code in decrypted_ascii_message]
        decrypted_message = ''.join(chr(ascii_code) for ascii_code in decrypted_ascii_message)
        self.decrypted_message = decrypted_message

        print(f"\nMessage has been decrypted to '{decrypted_message}'")


    def QuantumDecryption(self):
        """
            Add description
        """

        print("\nUsing the magical powers of quantum computation to decrypt the message ...")
        encrypted_message = self.encrypted_message
        self.deciphered_ascii_message = []
        self.deciphered_message = []
        for b in tqdm(encrypted_message):
            self._initialize_quantum_curcuit(b)
            self.found_period = False
            loop_counter = 0
            while not self.found_period and counter <= 30:
                self._measure_quantum_curcuit()
                self._period_finding_algorithm(b)
                loop_counter += 1
            if self.found_period:
                self._retrieve_message(b)
            else:
                self.deciphered_message.append("?")

        print(f"\n..Done. The message was deciphered to {''.join(self.deciphered_message)}")

    def _initialize_quantum_curcuit(self,b):
        """
            Add description
        """

        n = self.n
        N = self.N

        qreg = QuantumRegister(n, name="qreg")
        creg = ClassicalRegister(n, name="creg")
        qc = QuantumCircuit(qreg,creg)

        if self.verbose:
            print("\n   Running pre-calculations for quantum curcuit initialization ...")
        # compute \Sum_{x=0}^{2^n} f(x) where f(x) = b^x (mod N) and assign to output registry
        output_registry_state = np.zeros(2**n)
        previous_b = 1
        for input_registry_int in range(2**n):
            previous_b = (previous_b * b) % N
            output_registry_state[input_registry_int] = previous_b
        if self.verbose:
            print("   ..Done")
    
        if self.verbose:
            print("   Initializing quantum curcuit ...")
        # perform measurement on output registry
        output_registry_state_measurement = int(np.random.choice(output_registry_state))
        # set input registry to superposition corresponding the the measurement performed on the output registry above
        input_registry_state = np.where(output_registry_state == output_registry_state_measurement, 1, 0)
        input_registry_state = input_registry_state / np.linalg.norm(input_registry_state)

        # initialize qiskit quantum curcuit 
        qc.initialize(input_registry_state, qreg[:])
        #self.psi = Statevector(qc)
        if self.verbose:
            print("   ..Done")

        # apply gates to curcuit
        for i in range(n-1,-1,-1):
            qc.h(qreg[i])
            qc.measure(qreg[i],creg[i])
            for j in range(i-1,-1,-1):
                qc.p(np.pi/(2**(abs(i-j))),j).c_if(creg[i],1)

        self.qc = qc
    
    def _measure_quantum_curcuit(self):
        """
            Add description
        """

        qc = self.qc

        simulator = AerSimulator()
        compiled_curcuit = transpile(qc, simulator)
        if self.verbose:
            print("   Measuring quantum curcuit ...")
        job = simulator.run(compiled_curcuit, shots=1)
        sim_result = job.result()
        counts = sim_result.get_counts()
        self.result = sim_result
        self.counts = counts


        y_bn = list(counts.keys())[0]
        y = bin_to_int(y_bn)
        self.y = y
        if self.verbose:
            print(f"   ..Done. y was measured to {y}")


    def _measure_quantum_curcuit_ibm(self):
        print(1)
        qc = self.qc

        # fetch the backend
        print(2)
        backend_name = "ibm_brisbane"
        print(3)
        backend = service.backend(backend_name)

        # create a pass manager, which will populate the method with 
        # everything that needs to be know about the backend, e.g. connectivity,
        # native gates
        pass_manager = generate_preset_pass_manager(optimization_level=3, backend=backend)

        # transpile the code (source code to source code)
        qc_transpiled = pass_manager.run(qc)
        pass_manager = generate_preset_pass_manager(optimization_level=1, backend=backend)
        print(4)
        #qc_transpiled = transpile(qc, backend)
        qc_transpiled = pass_manager.run(qc)


        # options = EstimatorOptions()
        # # level 1 = use measurement readout mitigation, level 2 = zero noise extrapolation, ... 
        # options.resilience_level = 1
        # # transpolation already done on local device
        # options.optimization_level = 0
        # # qc has a lot idle time, use dynamic coupling
        # options.dynamical_decoupling.enable = True
        # options.dynamical_decoupling.sequence_type = "XY4"

        # generate estimator object with options
        #estimator = EstimatorV2(backend, options)

        # run job with transpiled qc and opterator list
        print(5)
        job = backend.run(qc_transpiled, shots=10)
        print(6)
        job_id = job.job_id()
        print(7)
        self.job_id = job_id
        print("\njob_id", job_id)

    def _check_job_ibm(self):
        job = service.job(self.job_id)
        result = job.result()
        counts = result.get_counts()
        self.counts = counts
        print("\ncounts", counts)


        try:
            for i in range(10):
                y_bn = list(counts.keys())[i]
                y = bin_to_int(y_bn)
                print(f"\n{i}: y was measured to {y}")
        except:
            pass

        y_bn = list(counts.keys())[0]
        y = bin_to_int(y_bn)
        self.y = y
        print(f"\n..Done. y was measured to {y}")


    def _test_periodic_function(self,b,x):
        N = self.N
        return (b**x) % N

    
    def _period_finding_algorithm(self, b):
        # Initialize the input registry to the encrypted message b. Apply the quantum curcuit defined
        # above and make a few measurements and convert from binary to decimal. Each result will with
        # prob 40% be within 1/2 of a multiple of 2**n/r. n is known, calculate r within the error margin.
        #  Make a plot of different r. Take the closest integerhos

        n = self.n
        y = self.y

        j_over_r_approx_float = y/2**n
        if self.verbose:
            print(f"   denominator should be less than N {self.N} or {2**(n/2)}")
            print("   j_over_r_approx_float", j_over_r_approx_float)

        # compute continued fraction terms
        continued_fractions_list = []
        frac_part = j_over_r_approx_float
        counter = 0
        # FIXME
        counter_limit = 100
        while (frac_part > 1/2**n) and counter <= counter_limit:
            inv = 1/frac_part
            int_part = int(inv)
            frac_part = inv - int_part
            continued_fractions_list.append(int_part)
            counter+=1
        # retry if continued fractions stopped before finding largest possible denominator smaller than N
        if counter == counter_limit:
            if self.verbose:
                print(f"   continued fractions failed. Denominator {frac_part} needs to be less than {1/2**n}")
            return
        # retry if continued fractionas was unable to find a denominator which is less than N
        if len(continued_fractions_list) < 1:
            if self.verbose:
                print("   no period candidates found, retrying")
            return
        
        # using computed continued fractions terms above, compute the continued fraction which
        # has denominator less than 2**n
        j_over_r_approx_frac_temp = Fraction(-999)
        i=1
        while (i <= len(continued_fractions_list) and j_over_r_approx_frac_temp.denominator < self.N):
            j_over_r_approx_frac = j_over_r_approx_frac_temp
            continued_fractions_list_sliced = continued_fractions_list[:i]
            res = continued_fractions_list_sliced[-1]
            for int_part in reversed(continued_fractions_list_sliced[:-1]):
                res = int_part + Fraction(1,res)
            j_over_r_approx_frac_temp = Fraction(1,res)
            i+=1
            
        # check if j/r fraction matches the j/r float
        if np.round(float(j_over_r_approx_frac),3) == np.round(j_over_r_approx_float,3):
            if self.verbose:
                print(f"   successfully found j/r to {j_over_r_approx_frac}")
        else:
            if self.verbose:
                print(f"   failed to converge j_over_r_approx_float {j_over_r_approx_float} to j_over_r_approx_frac {j_over_r_approx_frac}")
            return

        r_0 = j_over_r_approx_frac.denominator
        r_list = [i for i in range(r_0, 2**(int(n/2)), r_0)]


        if self.verbose:
            print(f"   found {len(r_list)} period candidates")

        if len(r_list) > 30:
            if self.verbose:
                print("   too many period candidates, retrying")
            return
        
        for r_candidate in r_list:
            if b == self._test_periodic_function(b,1+r_candidate):
                self.r = r_candidate
                self.found_period = True
                #print(f"\nfound period r={self.r}")
                break
        
        if not self.found_period and self.verbose:
            print("   no match for potential period, retrying")
    
    def _retrieve_message(self, b):
        r = self.r
        c = self.encoding_key
        N = self.N

        self.retrieved_message = False
        it = N+1
        found_d_prime = False
        for d_prime in tqdm(range(0,it)):
            if c * d_prime % r == 1:
                self.d_prime = d_prime
                found_d_prime = True
                # print(f"found d prime {d_prime}")
                # print(f"c {c}")
                break
        
        if not found_d_prime:
            if self.verbose:
                print("   did not find d prime")
            self.deciphered_message.append("?")
            return
        
        deciphered_ascii_code = b**d_prime % N
        self.deciphered_ascii_message.append(deciphered_ascii_code)
        self.deciphered_message.append(''.join(chr(deciphered_ascii_code)))
        



    

        


my_class = QDSimulator(lower_bound=int(3e1), 
                       upper_bound=int(7e1),
                        message="Hello",
                        num_Qbits = 20,
                        verbose = False
                        )


my_class.InitializeEncryptionKeys()
my_class.EncryptMessage()
my_class.DecryptMessage()



QDSimulator intitialized!

Generated random prime numbers p=43 and q=53, and their product N=2279

Encoding key generated c=185 (with (p-1)*(q-1)=2184)

Generating decoding key ...


  7%|▋         | 149/2279 [00:00<00:00, 3198.12it/s]

..Done. Check pased. Decoding key d=425

Message has been encrypted to [349, 814, 1317, 1317, 1730]

Message has been decrypted to 'Hello'


In [3]:
# my_class._initialize_quantum_curcuit()
# my_class._measure_quantum_curcuit()
# my_class._period_finding_algorithm()

my_class.somefunc()


#my_class._period_finding_algorithm()

#my_class._initialize_quantum_curcuit()
#psi = my_class.psi
#qc = my_class.qc
#psi.draw("latex")
#qc.draw(output="mpl")


Using the magical powers of quantum computation to decrypt the message ...


100%|██████████| 5/5 [03:06<00:00, 37.26s/it]


..Done. The message was deciphered to Hello


In [ ]:
a = 707 * -997 + 1958*360
print(a)

In [ ]:
my_class._period_finding_algorithm()

In [ ]:
def func(b,x,N):
    return (b**x) % N


# b = 358

# mod_N = 779

# for i in range(10):
#     x = 85*i
#     a = func(b,x,mod_N)
#     print(x,a)
#     if a == b:
#         print("!!!!!!!!")

In [ ]:
def period_finding_func():
    mod_N = 779
    b = 358
    #mod_N = 4171
    #b = 220
    for pot_r in range(mod_N):
        f = func(b,pot_r,mod_N)
        if b==f:
            print("FOUND IT")
            print(pot_r)


period_finding_func()

# r=672

            
#b = 358
#mod_N = 779    
# mod_N = 4171
# b = 220 

# a = func(b,1+11*672,mod_N)
# print(a)

In [ ]:
# n = 20
# y = 169934
# r = 360

n = 14
y = 11490
r = 57

a = (y / 2**n) * r

print(a)

In [ ]:
n=5

qreg = QuantumRegister(n, name="qreg")
creg = ClassicalRegister(n, name="creg")
qc = QuantumCircuit(qreg,creg)


input_registry_state = [0] * 2**n
input_registry_state[3] = 1


qc.initialize(input_registry_state, qreg[:])

for i in range(n-1,-1,-1):
    qc.h(qreg[i])
    qc.measure(qreg[i],creg[i])
    for j in range(i-1,-1,-1):
        qc.p(np.pi/(2**(abs(i-j))),j).c_if(creg[i],1)


qc.draw(output="mpl")



service = QiskitRuntimeService()

        # fetch the backend
backend_name = "ibm_brisbane"
backend = service.backend(backend_name)


qc_transpiled = transpile(qc, backend)



job = backend.run(qc_transpiled, shots=10)

# job_id = job.job_id()
# print("\njob_id", job_id)

# result = job.result()
# counts = result.get_counts()
# print("\ncounts", counts)


# try:
#     for i in range(10):
#         y_bn = list(counts.keys())[i]
#         y = bin_to_int(y_bn)
#         print(f"\n{i}: y was measured to {y}")
# except:
#     pass



In [20]:
2**10


1024